In [1]:
import torch
from tqdm import tqdm
from model import seedformer_dim128
from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d.loss import chamfer_distance
import numpy as np
from torch.optim.lr_scheduler import StepLR
import utils.utils as utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Below is the implementation of custom dataset/dataloader. Additionally there is a collate function. The number of points in eaach pointcloud is 
different. In order to make them equal collate function is defined.

In [2]:
class RacingDataset(Dataset):
    def __init__(self,root_dir,target_points=2990):#4731
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()
        self.target_points = target_points

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)

        return points,pcd_path

   
    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=0:
                filtered_list.append(filename)
        return filtered_list
   


In [3]:

# Load the largest point cloud
pcd_pad = o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)

# Create the dataset and dataloader
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real", target_points=len(pcd_pad_tens))
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=8, collate_fn=utils.collate_fn)

print(len(dataloader))

/tmp/ipykernel_114475/1991080591.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)


750


# Training

In [4]:
torch.manual_seed(42)
#del model
model=seedformer_dim128(up_factors=[1, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
epochs=100
optimizer=torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # Reduce LR by a factor of 0.1 every 30 epochs

for epoch in range(epochs):
    running_loss=0
    # Wrap the DataLoader with tqdm to track progress
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)  # Move data to GPU if available

            optimizer.zero_grad()
            outputs = model(inputs)
            losses = []

            for input_pc, output_pc in zip(inputs, outputs):
                # Calculate Chamfer Distance loss using pytorch3d.loss.chamfer_distance
                loss, _ = chamfer_distance(input_pc.unsqueeze(0), output_pc.unsqueeze(0))
                losses.append(loss)

            loss = torch.mean(torch.stack(losses))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))  # Update tqdm progress bar with the current loss
        scheduler.step()  # Step the learning rate scheduler

    print(f'Epoch {epoch+1} Loss: {running_loss / len(dataloader)}')  # Log the epoch loss

print('Finished Training')


cpu


/home/omar/TUM/Masterarbeit/Codes/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Epoch 1/100:   0%|          | 0/750 [00:00<?, ?batch/s]

sample xyz.shape  torch.Size([8, 3, 4731])
sample npoints: -   2000
sampled_xyz
layer1 shape:  torch.Size([8, 128, 2000])
sample xyz.shape  torch.Size([8, 3, 2000])
sample npoints: -   1000
sampled_xyz


Epoch 1/100:   0%|          | 0/750 [00:01<?, ?batch/s]

layer2 shape:  torch.Size([8, 256, 1000])
sample xyz.shape  torch.Size([8, 3, 1000])
sample npoints: -   None
sampled_xyz


RuntimeError: Given groups=1, weight of size [512, 131, 1, 1], expected input[8, 259, 1, 1000] to have 131 channels, but got 259 channels instead

## Save The Data


In [ ]:
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=utils.collate_fn)

# Load real dataset
real_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
real_dataloader = DataLoader(real_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=utils.collate_fn)
utils.apply_and_save_res(dataset=real_dataset,dataloader=real_dataloader,model=model,savedir="/home/omar/TUM/Data/reconstructed_cropped/real")

## "Stitch" the data

## Create and save the new dataset

# Correcting Labels


In [ ]:
labels_folder="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label/"
save_folder="/home/omar/TUM/Data/SeedFormer_2602_npy/sim/labels/"
for label in os.listdir(labels_folder):
    file = open(labels_folder+label)
    bbox=file.read()
    bbox_correct=utils.correct_bbox_label(bbox)
    file.close()
    file_write=open(save_folder+label,"w+")
    file_write.write(" ".join(bbox_correct))
    file_write.close()
    print(bbox_correct)
    #print(items)

In [ ]:
reconstructed_car_path="/home/omar/TUM/Data/combined/sim/"
#bbox_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label"

#original_sim_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/pcl/"
original_sim_path="/home/omar/TUM/Data/reconstructed_scene/sim/"
#reconstructed_scene_path="/home/omar/TUM/Data/reconstructed_scene/sim/"
npy_final_path="/home/omar/TUM/Data/SeedFormer_2602_npy_zyx/reconstructed/points/"
file_car_list=[]
for filename_car in os.listdir(reconstructed_car_path):
    file_car_list.append(filename_car)
    txt=filename_car.replace('.pcd',".txt")
   # bbox = (open(bbox_path+"/"+txt, "r")).read().split()
    original_pcd = o3d.io.read_point_cloud(original_sim_path+filename_car)
    filename_=filename_car.split(".")[0]
    #if  (int)(filename_)<=24995:#<>
        
        #print(filename_)
    np_array=np.asarray(original_pcd.points)#.pcd
    #arr2d=np.asarray(pcd_sim.points)
    x=np_array.copy()[:,0]
    np_array[:,0]=np_array[:,2]
    np_array[:,2]=x
    np.save(npy_final_path+filename_,np_array)#.npy

    #reconstructed_car= o3d.io.read_point_cloud(reconstructed_car_path +"/"+ filename_car)
    #crop_invert=crop_invert_stitch(original_pcd,reconstructed_car, bbox)
    #o3d.io.write_point_cloud(reconstructed_scene_path+"/"+filename_car,crop_invert)
    #np_array=np.asarray(crop_invert.points)
    #np.save(npy_final_path+filename_car,np_array)
#



In [ ]:
#source_label_path = "/home/omar/TUM/Masterarbeit/Data/m1695833/Sim2RealDistributionAlignedDataset/real/data/label/"
#pcd_real=o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
#pcd_sim=o3d.io.read_point_cloud("/home/omar/TUM/Data/reconstructed_scene/sim/018845.pcd")
#arr=np.asarray(pcd_sim.points)
#print(pcd_sim.points)
#print("np shape: " ,np.asarray(pcd_sim.points).shape)
#arr2d=np.asarray(pcd_sim.points)
#x=arr2d.copy()[:,0]
#arr2d[:,0]=arr2d[:,2]
#arr2d[:,2]=x
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(arr2d)
#o3d.io.write_point_cloud("/home/omar/TUM/Data/sync.pcd", pcd)
#
## Load saved point cloud and visualize it
#pcd_load = o3d.io.read_point_cloud("/home/omar/TUM/Data/sync.pcd")
##o3d.visualization.draw_geometries([pcd_load])
#pcd_car_recons=o3d.io.read_point_cloud("/home/omar/TUM/Data/reconstructed_cropped/sim/020000.pcd")
#pcl=o3d.io.read_point_cloud("/home/omar/TUM/Data/SeedFormer_2602_npy_zyx/real/points/018840.npy")
pcl=np.load("/home/omar/TUM/Data/SeedFormer_2602_npy/sim/points/015500.npy")
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
#box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,1.56 ,6.55 ,1.01 ,-0.0]
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,76.79 ,1.53,0.96 ,0.06]

#pcd_cut=crop_invert_stitch(pcl,box)
#pcd=pcd_cut+pcd_car_recons
utils.visualize(pcl=pcd,bbox_coordinates=box)
#utils.visualize(pcl=pcd_sim)
#
#utils.visualize(pcl=pcd_sim)
#
#utils.visualize(pcl=pcl)
#
#utils.visualize(pcl=pcd)

In [ ]:
#pcl=np.load("/home/omar/TUM/Data/SeedFormer_2602_npy/reconstructed/points/018840.npy")
pcd=o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/real/018840.pcd")
print(len(pcd.points))
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(pcl)
#box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,1.56 ,6.55 ,1.01 ,-0.0]
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,81.49 ,2.12,0.76 ,0.06]

#pcd_cut=crop_invert_stitch(pcl,box)
#pcd=pcd_cut+pcd_car_recons
utils.visualize(pcl=pcd,bbox_coordinates=box)
#utils.visualize(pcl=pcd_sim)

In [ ]:
array2d=np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9],[10, 11, 12]])
print("array2d : ",array2d)
print("array2d shape " , array2d.shape)
print("-------------")
#print("transpose : " ,array2d.T)
#print("reshape:  " ,array2d.reshape(-1).T)
x=array2d.copy()[:,0]
print("x col : " ,array2d[:,0])

print("z col : " ,array2d[:,2])
array2d[:,0]=array2d[:,2]
print("x",x)
array2d[:,2]=x
print("x col  after swap: " ,array2d)

In [ ]:
# Load simulation dataset
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)

# Load real dataset
real_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
real_dataloader = DataLoader(real_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)

# Select a subset of samples
num_samples = 3

# Apply the model and visualize differences
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

for dataset, dataloader, name in [(simulation_dataset, simulation_dataloader, "Simulation"), 
                                  (real_dataset, real_dataloader, "Real")]:
    for i, data in enumerate(tqdm(dataloader, desc=f'Processing {name} dataset', unit='point cloud')):
        if i >= num_samples:
            break
        
        inputs, paths = data
        #print(paths)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)

        original_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(inputs.squeeze().cpu().numpy()))
        original_pc.paint_uniform_color([1, 0, 0])  # Paint original point cloud in red

        reconstructed_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(outputs.squeeze().cpu().numpy()))
        reconstructed_pc.paint_uniform_color([0, 1, 0])  # Paint reconstructed point cloud in green

        o3d.visualization.draw_geometries([original_pc, reconstructed_pc], window_name=f"{name} - Sample {i+1}")
#model=None

In [ ]:
# Notes#
'''
input threshold adjust - 
'''